In [1]:
# packs
import os
import fnmatch
import numpy as np
import pandas as pd
from google.cloud import bigquery
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager

#funciones
def consulta_archivo(patron, ruta='../querys'):
    """
    Ejecuta una consulta en BigQuery utilizando un archivo SQL que coincide con un patrón.

    Parameters:
    - patron (str): El patrón para buscar el archivo SQL.
    - ruta (str, optional): La ruta del directorio donde se buscarán los archivos SQL. 
                           Por defecto, es '../querys'.

    Returns:
    - pandas.DataFrame or None: El resultado de la consulta en forma de DataFrame de pandas. 
                               Si no se encuentra el archivo SQL, retorna None.

    Example:
    ```python
    patron_busqueda = 'megabrands'
    df_resultado = consulta_archivo(patron_busqueda)
    ```

    """
    # Encuentra el archivo SQL utilizando el patrón
    for nombre_archivo in os.listdir(ruta):
        if fnmatch.fnmatch(nombre_archivo, f'*{patron}*.sql'):
            # Construye la ruta completa al archivo SQL
            ruta_completa_sql = os.path.join(ruta, nombre_archivo)
            print(nombre_archivo)
            # Lee el contenido del archivo SQL
            with open(ruta_completa_sql, "r") as file:
                sql = file.read()

            # Ejecuta la consulta y convierte los resultados a un DataFrame de pandas
            cliente = bigquery.Client()
            df_resultado = cliente.query(sql).to_dataframe()
            print(df_resultado.shape)
            print(df_resultado.head(2))
            return df_resultado

    # Si no se encuentra ningún archivo que coincida con el patrón
    print('No se encontró el archivo SQL.')
    return None


def plot_percentiles(df, column_name, percentile_limit=100):
    """
    Crea un histograma con líneas punteadas para los percentiles 33.33% y 66.67%.

    Parameters:
    - df (pandas.DataFrame): El DataFrame que contiene la columna de interés.
    - column_name (str): El nombre de la columna para la cual se calcularán los percentiles y se creará el histograma.
    - percentile_limit (float, optional): El percentil límite para filtrar los valores del DataFrame. Por defecto, es 100.

    Returns:
    - None: Muestra el histograma con las líneas punteadas.

    Example:
    ```python
    plot_percentiles(df, 'last_interaction_days', 100)
    ```

    """
    # Calcula el percentil límite
    percentile_value = np.percentile(df[column_name], percentile_limit)


    # Crea el subset excluyendo los valores superiores al percentil límite
    df_subset = df[df[column_name] <= percentile_value]

    # Calcula el histograma
    hist, bins = np.histogram(df_subset[column_name], bins=20)

    # Calcula percentiles
    percentiles = np.percentile(df_subset[column_name], [33.33, 66.67])
    print(percentiles)

    # Crea el histograma
    plt.hist(df_subset[column_name], bins=20, edgecolor='black', color=(255/255, 192/255, 0))
    plt.axvline(percentiles[0], color='black', linestyle='dashed', linewidth=2, label='33.33% Percentile')
    plt.axvline(percentiles[1], color='black', linestyle='dashed', linewidth=2, label='66.67% Percentile')

    # Añade etiquetas y título
    plt.xlabel('Valor')
    plt.ylabel('Frecuencia')
    plt.title(f'Histograma y Percentiles ({column_name})')
    plt.legend()

    # Muestra el histograma
    plt.show()

# Ejemplo de uso con un DataFrame y una columna cualquiera
# plot_percentiles(df, 'tu_columna', 99)

# Llama a la función con el nombre de tu archivo SQL
# df_L2 = consulta_archivo('total')
# plot_percentiles(df_L2, 'last_interaction_days')



In [2]:
# Llama a la función con el nombre de tu archivo SQL
df_L1 = consulta_archivo('nes_mega')
# plot_percentiles(df_L1, 'Dias_interaccion')
print(df_L1[df_L1['nInteracciones'] == 0].shape)
plot_percentiles(df_L1, 'nInteracciones', 95)

interacciones_mega.sql
(1040478, 3)
                                              td_id1  nInteracciones  \
0  8af64c29b77b1f8f5d2240eef27631563de5cbc0adceeb...               1   
1  49735ca5fb550ca4203782fabb0cb4214402a6cdd86c13...               1   

   Dias_interaccion  
0               189  
1               189  


In [10]:
percentile_value_max = np.percentile(df_L1['nInteracciones'], 95)
df = df_L1[(df_L1['nInteracciones'] > 0) & (df_L1['nInteracciones'] < percentile_value_max)].copy()
print(df.shape)
percentile_value_max = np.percentile(df['nInteracciones'], 66)
print(df[df['nInteracciones'] > percentile_value_max].shape)
# Crear un nuevo DataFrame con valores menores o iguales a 1
menores1 = df[df['nInteracciones'] <= 1].copy()
print(menores1.shape)
# Excluir los valores menores o iguales a 1 del DataFrame original
df_final = df[df['nInteracciones'] > 1].copy()
print(df_final.shape)

(986789, 3)

In [ ]:
plot_percentiles(df_final, 'nInteracciones')

In [8]:
 # Calcula el percentil límite
# percentile_value = 
# np.percentile(df_final['nInteracciones'], 66)
# print(percentile_value)
percentile_value_max = np.percentile(df_final['nInteracciones'], 66)
print(df_final[df_final['nInteracciones'] > percentile_value_max].shape)
 # Calcula el percentil límite
percentile_value = np.percentile(df_L1['nInteracciones'], 95)
print(percentile_value)

(156549, 3)

In [ ]:
df.shape

In [ ]:
df[df['nInteracciones'] <= 2].shape

In [ ]:
# Calcular deciles para ambas variables
df_final['corte_Dias_interaccion'] = pd.qcut(df_final['Dias_interaccion'], q=5, labels=False, duplicates='drop')
df_final['corte_nInteracciones'] = pd.qcut(df_final['nInteracciones'], q=5, labels=False, duplicates='drop')
# Crear una tabla de contingencia (matriz de cruces)
contingency_table = pd.crosstab(df_final['corte_Dias_interaccion'], df_final['corte_nInteracciones'])

# Imprimir la tabla de contingencia
print(contingency_table)

In [ ]:
df_final.head()

In [ ]:
df_L1.head(2)

In [ ]:
df_L1.head(2)

In [ ]:
# Crear un mapa de calor con Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(contingency_table, annot=True, cmap="YlGnBu", fmt='g', cbar_kws={'label': 'Frecuencia'})
plt.xlabel('Quintiles de Interacciones')
plt.ylabel('Quintiles de Dias Ultima Interaccion')
plt.title('Mapa de Calor de la Tabla de Contingencia')
plt.show()

In [ ]:
# Llama a la función con el nombre de tu archivo SQL
df_L1 = consulta_archivo('tada_total')
# plot_percentiles(df_L1, 'Dias_interaccion')

In [ ]:
plot_percentiles(df_L1, 'ordenes', 99)

In [ ]:
plot_percentiles(df_L1, 'money',99)

In [ ]:
percentile_value_max = np.percentile(df['ordenes'], 20)
df = [(df['ordenes'] < percentile_value_max)].copy()
df.shape

In [ ]:
percentile_value_max = np.percentile(df_L1['money'], 99)
df = df_L1[(df_L1['money'] > 0) & (df_L1['money'] < percentile_value_max)].copy()
df.shape

In [ ]:
percentile_value_max = np.percentile(df['ordenes'], 99)
df = df[(df['ordenes'] > 0) & (df['ordenes'] < percentile_value_max)].copy()
df.shape

In [ ]:
# Calcular deciles para ambas variables
df['corte_ordenes'] = pd.qcut(df['ordenes'], q=3, labels=False, duplicates='drop')
df['corte_money'] = pd.qcut(df['money'], q=3, labels=False, duplicates='drop')
# Crear una tabla de contingencia (matriz de cruces)
contingency_table = pd.crosstab(df['corte_ordenes'], df['corte_money'])

# Imprimir la tabla de contingencia
print(contingency_table)

In [ ]:
contingency_table.to_csv('test.csv', index=False)

In [ ]:
df['corte_money'].head()

In [ ]:
# Crear un mapa de calor con Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(contingency_table, annot=True, cmap="YlGnBu", fmt='g', cbar_kws={'label': 'Frecuencia'})
plt.xlabel('Quintiles de Interacciones')
plt.ylabel('Quintiles de Dias Ultima Interaccion')
plt.title('Mapa de Calor de la Tabla de Contingencia')
plt.show()